In [1]:
# Imports 
import numpy as np
import random
import visvis as vv
import scipy
import math 
import skimage
from skimage import measure
from numpy import sin, cos, sinh, cosh,arcsin,arccos, pi, exp,abs
from visvis import Point, Pointset
from scipy.spatial import Voronoi, voronoi_plot_2d, distance 

# Fractals 2D
axes = vv.gca()
    
    
def fractalization(vert1, length,angle,rad):
    pp = Pointset(3)
    # Initially the shape is only going to be a line
    coeff  =0.5
    
    if length > 0.1: 
        diff_ang = pi/2
        rad *=0.8  
        # Draw the shape from t
        x_n = vert1[0] + length*cos(angle) 
        y_n = vert1[1] + length*sin(angle)
        z_n = vert1[2]
        vert2 = Point(x_n,y_n,z_n)
        
        # add lines
        pp.append(vert1); pp.append(vert2);
        #l = vv.Line(axes, pp)
        vv.solidLine(pp, radius = rad)
        
        new_length = length*coeff
        fractalization(vert2, new_length,angle-diff_ang,rad)
        #fractalization(vert2, new_length,angle)
        fractalization(vert2, new_length,angle+diff_ang,rad)
        
length = 1
number = 4
vert = Point(0.0,0.0,0.0)
rad = 0.05 
for i in range(number):
    fractalization(vert, length, 2*pi/number*i,rad)
app = vv.use()
app.Run()

In [2]:
# Fractals 3D
axes = vv.gca()
edges = [] 
def create_line(pt1,pt2): 
    l = Pointset(3)
    l.append(pt1);l.append(pt2); 
    vv.Line(axes,l)

def sol_line(pt1,pt2,rad): 
    l = Pointset(3)
    l.append(pt1);l.append(pt2); 
    vv.solidLine(l, radius = rad)
    
def rotate_angle(vert1,vert2,vert3,theta):
    # Rodrigue Transformation matrix 
    u = np.array([vert2.x-vert1.x, vert2.y-vert1.y,vert2.z-vert1.z])
    u = u/np.linalg.norm(u)
    ux, uy, uz = u 
    
    C = cos(theta)
    S = sin(theta)
    t = 1-C
    
    # Transformation matrix
    Trans_Mat = np.array([[t*ux**2 + C, t*ux*uy-S*uz, t*ux*uz+S*uy],[t*ux*uy+S*uz, t*uy**2+C, t*uy*uz-S*ux],[t*ux*uz-S*uy,t*uy*uz+S*ux,t*uz**2+C]])
    return vert1+ np.dot(Trans_Mat,np.array([vert3.x-vert1.x,vert3.y-vert1.y, vert3.z-vert1.z]).T)
    
def fractalization(vert1, vert2, length,rad):  
    if length > 0.3: 
        # Define important variables
        coeff  =0.5
        coeff_rad = 0.8
        diff_ang = pi/5
        length *= coeff 
        rad *=coeff_rad 
        
        # Determine line equation
        l_eq = coeff*np.array([vert2.x-vert1.x,vert2.y- vert1.y,vert2.z-vert1.z])

        # Create the perpendicular line 
        tol = 0.1
        l_eq_per= np.array([-l_eq[1],l_eq[0],0])
        if(np.linalg.norm(l_eq_per)<tol):
            l_eq_per = np.array([length,0,0])
        
        ref_pt1 = Point(vert2 + l_eq)
        ref_pt2 = Point(vert2 + l_eq_per)
        
        # First rotation around the perpendicular rod  
        ref_pt = rotate_angle(vert2,ref_pt2,ref_pt1,diff_ang)        

        # Set number of rotations 
        rot = 4
        angle =2*pi/rot
        edges.append(vert1)
        edges.append(vert2)
        for  i in range(rot): 
            target_pt = Point(rotate_angle(vert2,ref_pt1,Point(ref_pt),angle*i))
            sol_line(vert2,target_pt,rad)
            #create_line(vert2,target_pt)

            fractalization(vert2, target_pt,length,rad)

        
            
length = 1
vert = Point(0.0,0.0,0.0)

N = 7
angle = 2*pi/N
rad = 0.03
for i in range(N):
    for j in range(int(N/2)):
        if ((i==0 or i ==N/2) and j>0):
            break
        else:
            x_n = vert[0] + length*sin(angle*i)*cos(angle*j) 
            y_n = vert[1] + length*sin(angle*i)*sin(angle*j)
            z_n = vert[2] + length*cos(angle*i)
            vert_new= Point(x_n,y_n,z_n)
            sol_line(vert,vert_new,rad)
            #create_line(vert,vert_new)
            fractalization(vert,vert_new, length,rad)

app = vv.use()
app.Run()